# Citi Bike Data Cleaning

### Bias: Data filtering in the dataset

> “This data has been processed to remove trips taken by staff, trips to/from test stations, and any trips under 60 seconds (likely false starts or users re-docking bikes).”

These biases (staff trips excluded and sub-60-second trips removed) **do not impact our analysis.**

## Imports, Settings and Data Loading

In [1]:
from __future__ import annotations

# Standard library
import math
from typing import Any, Dict, Optional, Tuple
from pathlib import Path

# Scientific computing
import numpy as np
import pandas as pd
import itertools
from scipy.signal import periodogram
import statsmodels
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

# Visualization
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.offsetbox import AnchoredText
import seaborn as sns

# Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

In [2]:
pd.set_option("display.max_rows", 20)      
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", 80)

In [3]:
#Constants: schema, dtypes, dirs

SEED = 42

EXPECTED_RAW_COLS = [
    "ride_id",
    "rideable_type",
    "started_at",
    "ended_at",
    "start_station_name",
    "start_station_id",
    "end_station_name",
    "end_station_id",
    "start_lat",
    "start_lng",
    "end_lat",
    "end_lng",
    "member_casual",
]

LOAD_DTYPES = {
    "ride_id": "string",
    "rideable_type": "category",
    "start_station_name": "string",
    "start_station_id": "string",
    "end_station_name": "string",
    "end_station_id": "string",
    "start_lat": "float32",
    "start_lng": "float32",
    "end_lat": "float32",
    "end_lng": "float32",
    "member_casual": "category",
}

DATETIME_COLS = ["started_at", "ended_at"]

# NYC-ish bounding box (deliberately wide but excludes obvious junk like 0,0 and far-away points)
NYC_BOUNDS = {
    "lat_min": 40.4,
    "lat_max": 41.0,
    "lng_min": -74.3,
    "lng_max": -73.6,
}

BASE_DIR = Path(r"..\data\citi_bike_raw\unpacked")
OUT_DIR  = Path(r"..\data\processed")

SAMPLE_FRAC = 0.03
sample_percent = int(round(SAMPLE_FRAC * 100))
OUT_PATH = OUT_DIR / f"citi_bike_sampled_{sample_percent}_percent.csv"

In [4]:
#Loading helpers: schema enforcement + caching

def sample_one_csv(
    csv_path: Path,
    expected_raw_cols: List[str],
    dtypes: Dict[str, str],
    datetime_cols: List[str],
    sample_frac: float,
    random_state: int,
    add_source_file: bool = True,
) -> pd.DataFrame:
    """Read one CSV with schema enforcement + reproducible sampling."""
    df = pd.read_csv(csv_path, dtype=dtypes, parse_dates=datetime_cols)
    df.columns = df.columns.str.strip()

    if list(df.columns) != expected_raw_cols:
        raise ValueError(
            f"Schema mismatch in {csv_path.name}\n"
            f"Expected: {expected_raw_cols}\n"
            f"Found:    {list(df.columns)}"
        )

    df = df.sample(frac=sample_frac, random_state=random_state)

    if add_source_file:
        df["source_file"] = csv_path.name

    return df


def load_or_create_sample(
    base_dir: Path,
    out_path: Path,
    sample_frac: float,
    random_state: int,
) -> pd.DataFrame:
    """Load cached sample or create a new one."""
    if out_path.exists():
        df = pd.read_csv(out_path, dtype=LOAD_DTYPES, parse_dates=DATETIME_COLS)
        return df

    csv_files = sorted(base_dir.rglob("*.csv"))
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found under {base_dir}")

    dfs = [
        sample_one_csv(
            csv_path=p,
            expected_raw_cols=EXPECTED_RAW_COLS,
            dtypes=LOAD_DTYPES,
            datetime_cols=DATETIME_COLS,
            sample_frac=sample_frac,
            random_state=random_state,
        )
        for p in csv_files
    ]

    df = pd.concat(dfs, ignore_index=True)
    df.to_csv(out_path, index=False)
    return df

- We exclude Jersey City trip files, as this analysis focuses exclusively on New York City data.  
  (The motor vehicle accident dataset is NYC-only, so including Jersey City would introduce geographic mismatch.)

- The `unpacked/` directory contains all extracted monthly Citi Bike datasets from  
  **202411-citibike-tripdata** through **202510-citibike-tripdata**.  
  The working DataFrame is created by sampling uniformly from this full time range.

In [5]:
#Load data

df_citibike = load_or_create_sample(
    base_dir=BASE_DIR,
    out_path=OUT_PATH,
    sample_frac=SAMPLE_FRAC,
    random_state=SEED,
)

## Overview

In [6]:
df_citibike.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
0,38658C64FCBE6EAD,electric_bike,2024-11-14 07:06:27.367,2024-11-14 07:10:59.644,Greenwich St & Hubert St,5470.10,Centre St & Chambers St,5207.01,40.721317,-74.010063,40.712734,-74.004608,member,202411-citibike-tripdata_1.csv
1,4B8FB1EE481B7B5D,classic_bike,2024-11-07 11:23:45.587,2024-11-07 11:29:55.879,Frederick Douglass Blvd & W 115 St,7658.13,E 106 St & Madison Ave,7528.31,40.803864,-73.955933,40.793434,-73.949448,member,202411-citibike-tripdata_1.csv
2,B2244A69E4D70034,classic_bike,2024-11-02 17:37:09.850,2024-11-02 17:48:37.132,W 20 St & 7 Ave,6182.02,King St & Varick St,5687.11,40.742390,-73.997261,40.727898,-74.005363,member,202411-citibike-tripdata_1.csv
3,0394AFE351677235,electric_bike,2024-11-04 18:09:08.203,2024-11-04 18:26:57.723,Harrison Pl & Porter Ave,5090.06,5 St & 51 Ave,6137.04,40.706860,-73.928513,40.742374,-73.956596,member,202411-citibike-tripdata_1.csv
4,8C84290C44C57A40,electric_bike,2024-11-10 15:07:32.908,2024-11-10 15:47:25.503,10 St & 7 Ave,3762.08,W 54 St & 6 Ave,6771.13,40.666206,-73.982002,40.761864,-73.977242,member,202411-citibike-tripdata_1.csv


In [7]:
df_citibike.shape

(1388512, 14)

In [8]:
df_citibike.describe()

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng
count,1388512,1388512,1.387862e+06,1.387862e+06,1.384437e+06,1.384437e+06
mean,2025-05-27 19:54:30.615766272,2025-05-27 20:07:22.012805632,4.073677e+01,-7.397169e+01,4.073653e+01,-7.397166e+01
min,2024-10-31 15:11:07.863000,2024-11-01 00:00:55.765000,4.062402e+01,-7.403962e+01,0.000000e+00,-7.407126e+01
25%,2025-03-15 07:48:09.245250048,2025-03-15 08:01:47.137499904,4.071269e+01,-7.399201e+01,4.071263e+01,-7.399209e+01
50%,2025-06-11 18:27:35.089999872,2025-06-11 18:40:30.166000128,4.073544e+01,-7.397907e+01,4.073537e+01,-7.397945e+01
75%,2025-08-22 13:34:01.775749888,2025-08-22 13:46:31.838249984,4.076018e+01,-7.395519e+01,4.076009e+01,-7.395520e+01
max,2025-10-31 23:54:48.606000,2025-10-31 23:59:30.796000,4.088630e+01,-7.384672e+01,4.088630e+01,0.000000e+00
std,NaN,NaN,4.853058e-01,1.140194e+00,4.882487e-01,1.140048e+00


In [9]:
# check dtype,null count/percentage,unique values and sample value

def audit_data_quality(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    n = len(df)
    for col in df.columns:
        null_count = int(df[col].isna().sum())
        rows.append(
            {
                "column": col,
                "dtype": str(df[col].dtype),
                "null_count": null_count,
                "null_pct": 100 * null_count / n,
                "unique_values": int(df[col].nunique(dropna=True)),
                "sample_values": df[col].dropna().head(3).tolist(),
            }
        )
    return pd.DataFrame(rows)

quality_report = audit_data_quality(df_citibike)
quality_report

,column,dtype,null_count,null_pct,unique_values,sample_values
0,ride_id,string,0,0.000000,1388512,"[38658C64FCBE6EAD, 4B8FB1EE481B7B5D, B2244A69E4D70034]"
1,rideable_type,category,0,0.000000,2,"[electric_bike, classic_bike, classic_bike]"
2,started_at,datetime64[ns],0,0.000000,1388453,"[2024-11-14 07:06:27.367000, 2024-11-07 11:23:45.587000, 2024-11-02 17:37:09..."
3,ended_at,datetime64[ns],0,0.000000,1388458,"[2024-11-14 07:10:59.644000, 2024-11-07 11:29:55.879000, 2024-11-02 17:48:37..."
4,start_station_name,string,650,0.046813,2215,"[Greenwich St & Hubert St, Frederick Douglass Blvd & W 115 St, W 20 St & 7 Ave]"
5,start_station_id,string,650,0.046813,2308,"[5470.10, 7658.13, 6182.02]"
6,end_station_name,string,3813,0.274611,2249,"[Centre St & Chambers St, E 106 St & Madison Ave, King St & Varick St]"
7,end_station_id,string,4075,0.293480,2334,"[5207.01, 7528.31, 5687.11]"
8,start_lat,float32,650,0.046813,2180,"[40.721317291259766, 40.803863525390625, 40.74238967895508]"
9,start_lng,float32,650,0.046813,2106,"[-74.01006317138672, -73.9559326171875, -73.99726104736328]"


In [10]:
#check duplicates

n_dup = df_citibike.duplicated(subset=["ride_id"]).sum()
print("Duplicate ride_id rows:", int(n_dup))
assert n_dup == 0, "Duplicate ride_id found (unexpected)."

Duplicate ride_id rows: 0


- data loaded and data types infered succesfully
- missing values only in the location (lng/lat/id/name) columns
- start/end time and ride id have (close to) as many values as rows (thats expected)
- no duplicates
- station id's have a unusual format (xxxx.xx), I expected them to be integers but it is confirmed on https://account.citibikenyc.com/map

## Missing Values

split rows with missing values into a seperate dataframe for later analysis

In [11]:
# rows with at least one missing value
mask_missing = df_citibike.isna().any(axis=1)

df_citibike_missing = df_citibike.loc[mask_missing].copy()
df_citibike = df_citibike.loc[~mask_missing].copy()

print("Rows with missing values:", df_citibike_missing.shape)
print("Rows without missing values:", df_citibike.shape)

Rows with missing values: (4578, 14)
Rows without missing values: (1383934, 14)


## Categorical Features

In [12]:
ride_type_table = (
    df_citibike["rideable_type"]
    .value_counts(dropna=False)
    .rename_axis("rideable_type")
    .reset_index(name="count")
)

ride_type_table["pct"] = 100 * ride_type_table["count"] / ride_type_table["count"].sum()
ride_type_table

,rideable_type,count,pct
0,electric_bike,968433,69.97682
1,classic_bike,415501,30.02318


In [13]:
customer_type_table = (
    df_citibike["member_casual"]
    .value_counts(dropna=False)
    .rename_axis("member_casual")
    .reset_index(name="count")
)

customer_type_table["pct"] = 100 * customer_type_table["count"] / customer_type_table["count"].sum()
customer_type_table

,member_casual,count,pct
0,member,1139505,82.338103
1,casual,244429,17.661897


- both categorical columns (rideable_type and costumer_type) have two unique values

## Time Features

In [14]:
#check time range
print("Date range started:", df_citibike["started_at"].min(), "→", df_citibike["started_at"].max())
print("Date range ended:  ", df_citibike["ended_at"].min(),   "→", df_citibike["ended_at"].max())

Date range started: 2024-10-31 15:11:07.863000 → 2025-10-31 23:54:48.606000
Date range ended:   2024-11-01 00:00:55.765000 → 2025-10-31 23:59:30.796000


In [15]:
#check rides which started in 2024-10-31
TARGET_DATE = pd.Timestamp("2024-10-31").date()
PREVIEW_N = 10

mask_started_1031 = df_citibike["started_at"].dt.date == TARGET_DATE

print(f"Rows with started_at on {TARGET_DATE}: {int(mask_started_1031.sum()):,}")
display(
    df_citibike.loc[mask_started_1031]
          .sort_values("started_at")
          .head(PREVIEW_N)
)

Rows with started_at on 2024-10-31: 32


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
12794,A33C5E2D8757B85D,electric_bike,2024-10-31 15:11:07.863,2024-11-01 02:15:40.584,Westchester Ave & Jackson Ave,7853.01,E 149 St & Jackson Ave,7812.01,40.816124,-73.908119,40.813133,-73.909233,member,202411-citibike-tripdata_1.csv
31202,A8B80A1231856B04,electric_bike,2024-10-31 22:33:19.683,2024-11-01 00:28:26.940,E 23 St & 1 Ave,5929.01,E 23 St & 1 Ave,5929.01,40.736504,-73.978096,40.736504,-73.978096,casual,202411-citibike-tripdata_2.csv
33498,1990CBF739C670F1,electric_bike,2024-10-31 23:26:01.671,2024-11-01 00:01:03.959,1 Ave & E 38 St,6230.02,Vesey St & Church St,5216.06,40.746201,-73.971825,40.712219,-74.010475,member,202411-citibike-tripdata_2.csv
29083,ABCE412DC31CC152,electric_bike,2024-10-31 23:29:19.070,2024-11-01 00:05:18.396,W 12 St & Hudson St,5997.10,W 43 St & 10 Ave,6756.01,40.737530,-74.005592,40.760094,-73.994621,member,202411-citibike-tripdata_1.csv
55682,9FA1CA2DDA669FCA,electric_bike,2024-10-31 23:29:59.666,2024-11-01 00:03:09.064,E 26 St & 3 Ave,6089.11,W 27 St & 7 Ave,6257.06,40.740692,-73.981606,40.746651,-73.993919,casual,202411-citibike-tripdata_2.csv
7380,6C2F713FC6F12337,classic_bike,2024-10-31 23:32:13.006,2024-11-01 00:06:11.116,Barrow St & Hudson St,5805.05,E 10 St & 2 Ave,5746.02,40.731724,-74.006744,40.729710,-73.986595,member,202411-citibike-tripdata_1.csv
54134,E819BF37CF1DF62D,electric_bike,2024-10-31 23:39:09.518,2024-11-01 00:08:23.416,George St & Wilson Ave,4906.07,3 Ave & Schermerhorn St,4437.01,40.702995,-73.929970,40.686832,-73.979675,casual,202411-citibike-tripdata_2.csv
57575,1D42AEF3D2032EAB,electric_bike,2024-10-31 23:41:20.960,2024-11-01 00:06:58.811,Kent Ave & N 7 St,5489.03,2 Ave & E 72 St,6925.09,40.720367,-73.961647,40.768761,-73.958405,member,202411-citibike-tripdata_2.csv
25427,69FAC951C0249CAF,electric_bike,2024-10-31 23:43:18.275,2024-11-01 00:00:55.765,S 4 St & Wythe Ave,5204.05,Bergen St & Kingston Ave,4089.01,40.712860,-73.965904,40.675598,-73.941498,member,202411-citibike-tripdata_1.csv
42700,67A56365766902D2,electric_bike,2024-10-31 23:44:22.123,2024-11-01 00:03:08.269,6 Ave & Walker St,5430.10,Milton St & Franklin St,5752.07,40.719791,-74.005226,40.729061,-73.957787,member,202411-citibike-tripdata_2.csv


In [16]:
# check if any of the ones which started in 
NEXT_DATE = TARGET_DATE + pd.Timedelta(days=1)

mask_ended_before_next_day = df_citibike["ended_at"].dt.date < NEXT_DATE

violations = df_citibike.loc[mask_started_1031 & mask_ended_before_next_day]

print(f"Violations (ended before {NEXT_DATE}): {len(violations):,}")

if violations.empty:
    print("✅ All rides starting on 2024-10-31 end on 2024-11-01 or later.")
else:
    display(
        violations.sort_values(["started_at", "ended_at"]).head(20)
    )

Violations (ended before 2024-11-01): 0
✅ All rides starting on 2024-10-31 end on 2024-11-01 or later.


- Even though the data was loaded for the period **2024-11 to 2025-10**, some trips have a `started_at` timestamp of **2024-10-31**.
- None of these trips ended on **2024-10-31**; all ended on **2024-11-01** or later.
- **Conclusion:** the data was most likely grouped by **end time**, which explains why trips with earlier start dates appear.

## Lat/Long Out of Bounds

In [17]:
# Rows with coordinates outside of NYC?

def out_of_bounds_mask(
    df: pd.DataFrame,
    side: str,
    bounds: dict = NYC_BOUNDS,
) -> pd.Series:
    """
    Boolean mask for rows where {side}_lat/{side}_lng are present but outside bounds.
    side ∈ {"start", "end"}
    """
    lat = f"{side}_lat"
    lng = f"{side}_lng"

    coord_present = df[lat].notna() & df[lng].notna()

    oob = (
        (df[lat] < bounds["lat_min"]) | (df[lat] > bounds["lat_max"]) |
        (df[lng] < bounds["lng_min"]) | (df[lng] > bounds["lng_max"])
    )

    return coord_present & oob


def summarize_mask(df: pd.DataFrame, mask: pd.Series, label: str) -> dict:
    total = len(df)
    count = int(mask.sum())
    pct = 100.0 * count / total if total else 0.0
    return {"issue": label, "count": count, "pct": pct}

In [18]:
start_oob = out_of_bounds_mask(df_citibike, "start")
end_oob = out_of_bounds_mask(df_citibike, "end")

summary_rows = [
    summarize_mask(df_citibike, start_oob, "Start out of bounds"),
    summarize_mask(df_citibike, end_oob, "End out of bounds"),
]

print("GEOGRAPHIC ANOMALIES: OUT OF BOUNDS")
print("=" * 60)
for r in summary_rows:
    print(f"{r['issue']:.<40} {r['count']:>8,} ({r['pct']:>6.2f}%)")
print("=" * 60)

GEOGRAPHIC ANOMALIES: OUT OF BOUNDS
Start out of bounds.....................        0 (  0.00%)
End out of bounds.......................        3 (  0.00%)


In [19]:
end_oob_rows = df_citibike.loc[end_oob].copy()
print(f"Rows with END coords out of bounds: {len(end_oob_rows)}")
display(end_oob_rows)

Rows with END coords out of bounds: 3


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
101085,4EC49D3092CFB329,classic_bike,2024-11-20 14:33:29.099,2024-11-20 14:39:42.336,Jefferson St & Cypress Ave,5082.08,Bronx WH station,SYS018,40.709068,-73.921570,0.0,0.0,member,202411-citibike-tripdata_4.csv
108765,C3AA4E02DACEDB79,electric_bike,2024-11-18 06:41:54.926,2024-11-18 06:55:55.178,New York Ave & St Marks Ave,4098.06,Bronx WH station,SYS018,40.675140,-73.947159,0.0,0.0,member,202411-citibike-tripdata_4.csv
126655,68F26C412D1D41ED,electric_bike,2024-12-09 06:48:09.410,2024-12-09 07:02:47.872,New York Ave & St Marks Ave,4098.06,Bronx WH station,SYS018,40.675140,-73.947159,0.0,0.0,member,202412-citibike-tripdata_1.csv


- This looks rides to a internal/maintanance station "Bronx WH station SYS018"(WH = Warehouse, SYS=System)
- look deeper into that

## Internal Stations

In [20]:
# finding internal stations

def maintenance_station_mask(df: pd.DataFrame) -> pd.Series:
    """
    Rows that look like internal maintenance/warehouse activity:
    - station name contains whole-word 'WH'
    - OR station id starts with 'SYS'
    """
    start_name = df["start_station_name"].astype("string")
    end_name   = df["end_station_name"].astype("string")
    start_id   = df["start_station_id"].astype("string")
    end_id     = df["end_station_id"].astype("string")

    has_wh = (
        start_name.str.contains(r"\bWH\b", regex=True, na=False) |
        end_name.str.contains(r"\bWH\b", regex=True, na=False)
    )
    has_sys = (
        start_id.str.startswith("SYS", na=False) |
        end_id.str.startswith("SYS", na=False)
    )
    return has_wh | has_sys


mask_maint = maintenance_station_mask(df_citibike)

print(f"Rows matching (WH in name OR SYS in id): {int(mask_maint.sum()):,} of {len(df_citibike):,}")

sys_ids = pd.unique(pd.concat([
    df_citibike.loc[df_citibike["start_station_id"].astype("string").str.startswith("SYS", na=False), "start_station_id"],
    df_citibike.loc[df_citibike["end_station_id"].astype("string").str.startswith("SYS", na=False), "end_station_id"],
]).astype("string"))

wh_names = pd.unique(pd.concat([
    df_citibike.loc[df_citibike["start_station_name"].astype("string").str.contains(r"\bWH\b", regex=True, na=False), "start_station_name"],
    df_citibike.loc[df_citibike["end_station_name"].astype("string").str.contains(r"\bWH\b", regex=True, na=False), "end_station_name"],
]).astype("string"))

print("Distinct SYS station IDs:", len(sys_ids))
print("Distinct WH station names:", len(wh_names))
print("\nSYS IDs:", list(sys_ids))
print("\nWH names:", list(wh_names))

display(
    df_citibike.loc[
        mask_maint,:]
)
df_citibike.loc[
        mask_maint,:].head(50)

Rows matching (WH in name OR SYS in id): 195 of 1,383,934
Distinct SYS station IDs: 4
Distinct WH station names: 1

SYS IDs: ['SYS016', 'SYS038', 'SYS033', 'SYS018']

WH names: ['Bronx WH station']


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
2961,DC8A58B41B46D99A,electric_bike,2024-11-04 06:44:47.754,2024-11-04 06:52:16.216,Bushwick Ave & Harman St,4640.01,Morgan Bike Mechanics,SYS016,40.693562,-73.925240,40.709629,-73.931458,member,202411-citibike-tripdata_1.csv
8047,E9979861B3EBCD5C,electric_bike,2024-11-11 07:06:16.923,2024-11-11 07:18:32.127,Morgan Bike Mechanics,SYS016,Park Ave & Marcus Garvey Blvd,4768.02,40.709629,-73.931458,40.698616,-73.941345,casual,202411-citibike-tripdata_1.csv
29658,C16F3175916D235E,electric_bike,2024-11-02 14:25:29.536,2024-11-02 14:48:22.760,Vesey St & Church St,5216.06,Morgan Bike Mechanics,SYS016,40.712219,-74.010475,40.709629,-73.931458,member,202411-citibike-tripdata_1.csv
37384,5A36F1C4C5E65D45,electric_bike,2024-11-06 23:02:16.570,2024-11-06 23:37:10.230,Morgan Bike Mechanics,SYS016,Schenectady Ave & Maple St,3639.04,40.709629,-73.931458,40.661781,-73.934448,member,202411-citibike-tripdata_2.csv
43570,52FD2F787A29DCDA,electric_bike,2024-11-05 06:40:53.673,2024-11-05 06:47:24.432,Bushwick Ave & Harman St,4640.01,Morgan Bike Mechanics,SYS016,40.693562,-73.925240,40.709629,-73.931458,member,202411-citibike-tripdata_2.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306173,3F3679225481EDB6,classic_bike,2025-10-13 14:26:52.285,2025-10-13 14:43:38.396,Montrose Ave & Bushwick Ave,5068.02,Morgan Bike Mechanics,SYS016,40.707680,-73.940163,40.709629,-73.931458,member,202510-citibike-tripdata_2.csv
1316201,F10DF1A4CB9E090E,electric_bike,2025-10-12 07:08:35.523,2025-10-12 07:24:19.746,Morgan Bike Mechanics,SYS016,St Johns Pl & Howard Ave,3917.03,40.709629,-73.931458,40.669922,-73.920021,member,202510-citibike-tripdata_3.csv
1318174,F53869AC4D1D8306,electric_bike,2025-10-17 06:58:56.924,2025-10-17 07:01:11.748,Greenwich St & W Houston St,5730.08,Pier 40 X2,SYS033,40.728848,-74.008591,40.728485,-74.011696,member,202510-citibike-tripdata_3.csv
1329562,10CC0BF4DBF5CAD3,electric_bike,2025-10-21 22:34:35.093,2025-10-21 22:53:40.897,S 4 St & Rodney St,5156.05,Pier 40 X2,SYS033,40.709339,-73.956078,40.728485,-74.011696,member,202510-citibike-tripdata_3.csv


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
2961,DC8A58B41B46D99A,electric_bike,2024-11-04 06:44:47.754,2024-11-04 06:52:16.216,Bushwick Ave & Harman St,4640.01,Morgan Bike Mechanics,SYS016,40.693562,-73.925240,40.709629,-73.931458,member,202411-citibike-tripdata_1.csv
8047,E9979861B3EBCD5C,electric_bike,2024-11-11 07:06:16.923,2024-11-11 07:18:32.127,Morgan Bike Mechanics,SYS016,Park Ave & Marcus Garvey Blvd,4768.02,40.709629,-73.931458,40.698616,-73.941345,casual,202411-citibike-tripdata_1.csv
29658,C16F3175916D235E,electric_bike,2024-11-02 14:25:29.536,2024-11-02 14:48:22.760,Vesey St & Church St,5216.06,Morgan Bike Mechanics,SYS016,40.712219,-74.010475,40.709629,-73.931458,member,202411-citibike-tripdata_1.csv
37384,5A36F1C4C5E65D45,electric_bike,2024-11-06 23:02:16.570,2024-11-06 23:37:10.230,Morgan Bike Mechanics,SYS016,Schenectady Ave & Maple St,3639.04,40.709629,-73.931458,40.661781,-73.934448,member,202411-citibike-tripdata_2.csv
43570,52FD2F787A29DCDA,electric_bike,2024-11-05 06:40:53.673,2024-11-05 06:47:24.432,Bushwick Ave & Harman St,4640.01,Morgan Bike Mechanics,SYS016,40.693562,-73.925240,40.709629,-73.931458,member,202411-citibike-tripdata_2.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300084,57CA508EAD474BA0,electric_bike,2025-02-27 06:53:13.216,2025-02-27 07:02:14.933,McKibbin St & Bogart St,5059.02,Morgan Bike Mechanics,SYS016,40.706238,-73.933868,40.709629,-73.931458,member,202502-citibike-tripdata_2.csv
306074,92F4D00421DFFCE7,electric_bike,2025-03-14 14:55:12.300,2025-03-14 15:41:45.337,Pier 40 X2,SYS033,Cliff St & Fulton St,5065.14,40.728485,-74.011696,40.708382,-74.004951,member,202503-citibike-tripdata_1.csv
319154,89B7428CB7B05D69,electric_bike,2025-03-10 07:06:50.152,2025-03-10 07:13:08.492,Carmine St & 6 Ave,5763.03,Pier 40 X2,SYS033,40.730385,-74.002151,40.728485,-74.011696,casual,202503-citibike-tripdata_1.csv
321044,E9E844DCE989E823,electric_bike,2025-03-13 14:46:39.735,2025-03-13 15:00:02.209,Bergen St & Kingston Ave,4089.01,Morgan Bike Mechanics,SYS016,40.675598,-73.941498,40.709629,-73.931458,member,202503-citibike-tripdata_1.csv


- "Morgan Loading Docks","Morgan Bike Mechanics","Pier 40 X2","Bronx WH station"
- these are all rides to or from ***maintanance stations*** <br>
-> drop them

In [21]:
rows_before = len(df_citibike)
df_citibike = df_citibike.loc[~mask_maint].copy()
rows_after = len(df_citibike)

print(f"Dropped {rows_before - rows_after:,} rows using maintenance mask")
print("New shape:", df_citibike.shape)

Dropped 195 rows using maintenance mask
New shape: (1383739, 14)


## Station ID

- station id should has format xxxx.xx

In [32]:
# pattern: exactly 4 digits, dot, 2 digits (e.g. 5470.10)
pat = r"^\d{4}\.\d{2}$"

# make one Series of all station IDs (start + end)
all_ids = pd.concat(
    [df_citibike["start_station_id"], df_citibike["end_station_id"]],
    ignore_index=True
).dropna().astype(str)

# station IDs that do NOT match iiii.ii
bad_ids = all_ids[~all_ids.str.match(pat)]

print("Total unique station IDs:", all_ids.nunique())
print("Unique IDs NOT matching iiii.ii:", bad_ids.nunique())

# show the actual IDs
display(pd.Series(bad_ids.unique()).sort_values().reset_index(drop=True))

Total unique station IDs: 2340
Unique IDs NOT matching iiii.ii: 134


0            3113.1
1            3391.1
2            3423.1
3            3576.1
4            3593.1
           ...     
129           JC104
130           JC110
131           JC115
132           JC116
133    Shop Morgan 
Length: 134, dtype: object

- 133 station ids which dont fit the format

Most common reason for invalid format: Missing trailing zeros

Suprisingly: There are 29 stations with the format JCxxx and HBxxx, which implies that there are trips to/from **Jersey City/Hopkins** stations in the dataset

In [23]:
# Look at trips to/from Hobkins or Jersey City
PREFIXES = ("JC", "HB")

def station_prefix_mask(df: pd.DataFrame, col: str, prefixes: tuple[str, ...]) -> pd.Series:
    s = df[col].astype("string")
    return s.str.startswith(prefixes, na=False)

start_mask = station_prefix_mask(df_citibike, "start_station_id", PREFIXES)
end_mask   = station_prefix_mask(df_citibike, "end_station_id", PREFIXES)

start_jc_hb = df_citibike.loc[start_mask].copy()
end_jc_hb   = df_citibike.loc[end_mask].copy()

print(f"Rows with start_station_id starting with {PREFIXES}: {len(start_jc_hb):,}")
display(start_jc_hb)

print(f"Rows with end_station_id starting with {PREFIXES}: {len(end_jc_hb):,}")
display(end_jc_hb)

Rows with start_station_id starting with ('JC', 'HB'): 0


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file


Rows with end_station_id starting with ('JC', 'HB'): 95


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
13287,ABAA2E2827C94E62,electric_bike,2024-11-06 18:19:35.622,2024-11-06 18:43:06.763,9 Ave & W 45 St,6717.06,14 St Ferry - 14 St & Shipyard Ln,HB202,40.760193,-73.991257,40.752960,-74.024353,member,202411-citibike-tripdata_1.csv
13698,4F8147F7E58F80EF,electric_bike,2024-11-06 17:20:52.966,2024-11-06 18:01:12.754,King St & Varick St,5687.11,Dey St,JC065,40.727898,-74.005363,40.737713,-74.066917,member,202411-citibike-tripdata_1.csv
13712,87CD2E2D84D1D7A4,electric_bike,2024-11-05 16:37:05.209,2024-11-05 16:59:25.585,E 43 St & Madison Ave,6551.11,14 St Ferry - 14 St & Shipyard Ln,HB202,40.753548,-73.978966,40.752960,-74.024353,member,202411-citibike-tripdata_1.csv
16532,64FE94EAE35D89E2,electric_bike,2024-11-02 19:53:43.944,2024-11-02 20:06:06.819,Vesey Pl & River Terrace,5297.02,Paulus Hook,JC002,40.715340,-74.016586,40.714146,-74.033554,member,202411-citibike-tripdata_1.csv
32843,304E5EB15156FBA5,electric_bike,2024-11-13 18:38:59.680,2024-11-13 19:10:51.724,Broadway & W 36 St,6441.01,11 St & Washington St,HB502,40.750977,-73.987656,40.749985,-74.027153,member,202411-citibike-tripdata_2.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265730,76F6571B3FF94B1A,electric_bike,2025-10-09 17:28:57.589,2025-10-09 18:08:34.852,W 16 St & The High Line,6233.05,Hamilton Park,JC009,40.743351,-74.006821,40.727596,-74.044250,member,202510-citibike-tripdata_1.csv
1328688,95F5C4DEDC5B725D,electric_bike,2025-10-27 19:32:27.624,2025-10-27 19:47:59.586,Watts St & Greenwich St,5578.02,Marin Light Rail,JC013,40.724056,-74.009659,40.714584,-74.042816,member,202510-citibike-tripdata_3.csv
1354834,71CE514502ADA9B6,classic_bike,2025-10-17 15:54:39.236,2025-10-17 16:33:33.899,W 33 St & 10 Ave,6492.04,Harborside,JC104,40.753773,-73.999374,40.719254,-74.034233,member,202510-citibike-tripdata_4.csv
1368834,A0BA0587908F1716,electric_bike,2025-10-23 14:45:24.654,2025-10-23 15:06:41.191,Vesey Pl & River Terrace,5297.02,Columbus Park - Clinton St & 9 St,HB501,40.715340,-74.016586,40.748161,-74.032455,member,202510-citibike-tripdata_5.csv


- Explanation: Only trips where the ***start station is in new york city*** are included, but ***end station can be arbitrary***
- ***Inconsistency***: This is exactly the other way around as for start/end time, there the end time needs to be in the valid time frame, but for station the start station needs to be in NYC. So a valid datapoint in our dataset could be e.g. a ride which start at 2024-10-31 in nyc and ends in 2024-11-01 in Jersey City, its a bit counter intuitiv but this sample would be in the 2024-11 citibike nyc dataset

## Station Ids/Names Incosistencies

In [24]:
start_name = df_citibike["start_station_name"]
end_name   = df_citibike["end_station_name"]
start_id   = df_citibike["start_station_id"]
end_id     = df_citibike["end_station_id"]

all_names = pd.concat([start_name, end_name], ignore_index=True)
all_ids   = pd.concat([start_id, end_id], ignore_index=True)

unique_station_stats = pd.DataFrame(
    {
        "column": [
            "start_station_name",
            "start_station_id",
            "end_station_name",
            "end_station_id",
            "ALL_station_names_combined",
            "ALL_station_ids_combined",
        ],
        "unique_values": [
            start_name.nunique(dropna=True),
            start_id.nunique(dropna=True),
            end_name.nunique(dropna=True),
            end_id.nunique(dropna=True),
            all_names.nunique(dropna=True),
            all_ids.nunique(dropna=True),
        ],
    }
)

display(unique_station_stats)

,column,unique_values
0,start_station_name,2212
1,start_station_id,2305
2,end_station_name,2245
3,end_station_id,2330
4,ALL_station_names_combined,2245
5,ALL_station_ids_combined,2340


- many inconsistencys
- interestingly, the offset between ALL_station_names_combined and ALL_station_ids_combined (95) is rouhgly the number of ids with invalid format (103 invalid ids if you don't count the 29 JC/HB stations)
- lets investigate

In [25]:
# Build unified (name, id) table from start + end
stations = pd.concat(
    [
        df_citibike[["start_station_name", "start_station_id"]]
            .rename(columns={"start_station_name": "name", "start_station_id": "id"}),
        df_citibike[["end_station_name", "end_station_id"]]
            .rename(columns={"end_station_name": "name", "end_station_id": "id"}),
    ],
    ignore_index=True,
)

# Count distinct IDs per station name
id_counts = stations.groupby("name")["id"].nunique()

# Names with more than one ID
ambiguous_names = id_counts[id_counts > 1].index

#row-level (name, id) pairs 
ambiguous_rows = (
    stations[stations["name"].isin(ambiguous_names)]
    .drop_duplicates()
    .sort_values(["name", "id"])
)

#Table 2: grouped table (name → list of IDs)
ambiguous_grouped = (
    ambiguous_rows
    .groupby("name")["id"]
    .apply(lambda x: sorted(x.unique()))
    .reset_index(name="ids")
)

print(f"Stations with >1 ID: {len(ambiguous_grouped)}")

display(ambiguous_grouped)

Stations with >1 ID: 105


,name,ids
0,11 St & 43 Ave,"[6438.04, 6438.05]"
1,2 Ave & E 99 St,"[7386.1, 7386.10]"
2,24 Ave & 26 St,"[7152.1, 7152.10]"
3,3 Ave & E 82 St,"[7154.1, 7154.10]"
4,3 St & Hoyt St,"[4110.1, 4110.10]"
...,...,...
100,W 47 St & 6 Ave,"[6667.04, 6667.12]"
101,W 51 St & 6 Ave,"[6740.1, 6740.10]"
102,W 74 St & Columbus Ave,"[7230.1, 7230.10]"
103,Walton Ave & E 171 St,"[8249.1, 8249.10]"


There are 3 problems
1. by far the biggest problem seem to be missing trailing zeros (7386.1, 7386.10)
2. both id's are valid but different (6438.04, 6438.05)
3. weird outliers which are not conform with naming convention (xxxx.xx) at all 

In [26]:
# Build unified (id, name) table from start + end
stations = (
    pd.concat(
        [
            df_citibike[["start_station_id", "start_station_name"]].rename(
                columns={"start_station_id": "id", "start_station_name": "name"}
            ),
            df_citibike[["end_station_id", "end_station_name"]].rename(
                columns={"end_station_id": "id", "end_station_name": "name"}
            ),
        ],
        ignore_index=True,
    )
)

# Station IDs with >1 name
ambiguous_grouped = (
    stations.drop_duplicates()
    .groupby("id", as_index=False)["name"]
    .agg(lambda s: sorted(s.unique()))
    .rename(columns={"name": "names"})
)

ambiguous_grouped = (
    ambiguous_grouped[ambiguous_grouped["names"].str.len() > 1]
    .sort_values("id", ignore_index=True)
)

print(f"Station IDs with >1 name: {len(ambiguous_grouped)}")
display(ambiguous_grouped)

Station IDs with >1 name: 11


,id,names
0,6505.01,"[24 St & 41 Ave, 24 St & 41 Ave (original)]"
1,6873.01,"[34 Ave & Vernon Blvd, 34th Ave & Vernon Blvd]"
2,7079.06,"[Central Park W & W 68 St, Central Park West & W 68 St]"
3,7141.07,"[Central Park W & W 72 St, Central Park West & W 72 St]"
4,7253.04,"[Central Park W & W 76 St, Central Park West & W 76 St]"
5,7304.08,"[W 82 St & Central Park W, W 82 St & Central Park West]"
6,7354.01,"[Central Park W & W 85 St, Central Park West & W 85 St]"
7,7606.01,"[W 106 St & Central Park W, W 106 St & Central Park West]"
8,7851.04,"[5 Av & W 139 St, 5 Ave & W 139 St]"
9,8226.07,"[W 168 St & Fort Washington Ave, W 168 St & Ft Washington Ave]"


- looks like the generell difference between the amount of (start/end) station names and station ids can be explained through changes in id (11 St & 43 Ave	[6438.04, 6438.05]) and assigning the same id with and without trailing zero (W 51 St & 6 Ave	[6740.1, 6740.10])

In [27]:
# station names which only appear as start/end of a trip

start_names = set(df_citibike["start_station_name"].dropna().unique())
end_names   = set(df_citibike["end_station_name"].dropna().unique())

# Names that appear only on one side
end_only_names   = end_names - start_names
start_only_names = start_names - end_names

print("Stations appearing only as END stations:", len(end_only_names))
print("Stations appearing only as START stations:", len(start_only_names))

# --- END-only table (IDs + counts) ---
end_only_table = (
    df_citibike.loc[df_citibike["end_station_name"].isin(end_only_names)]
    .groupby(["end_station_name", "end_station_id"])
    .size()
    .reset_index(name="n_rides_ended")
    .sort_values("n_rides_ended", ascending=False)
    .reset_index(drop=True)
)

# --- START-only table (IDs + counts) ---
start_only_table = (
    df_citibike.loc[df_citibike["start_station_name"].isin(start_only_names)]
    .groupby(["start_station_name", "start_station_id"])
    .size()
    .reset_index(name="n_rides_started")
    .sort_values("n_rides_started", ascending=False)
    .reset_index(drop=True)
)

display(end_only_table)
display(start_only_table)

Stations appearing only as END stations: 33
Stations appearing only as START stations: 0


,end_station_name,end_station_id,n_rides_ended
0,Morris Canal,JC072,18
1,14 St Ferry - 14 St & Shipyard Ln,HB202,13
2,Harborside,JC104,7
3,12 St & Sinatra Dr N,HB201,7
4,Marin Light Rail,JC013,6
...,...,...,...
28,Newport Pkwy,JC008,1
29,2 St & Park Ave,HB608,1
30,Ridge Blvd & 82 St,2463.04,1
31,Southwest Park - Jackson St & Observer Hwy,HB401,1


,start_station_name,start_station_id,n_rides_started


- the difference in start and end stations can be explained because the trip data includes trips which ended outside of NYC but not trips which start outside of NYC

## Create Unique Identifier

I have decided to use station name as the identifier because there are way more incosistencies in station id. So station name is already better and also easier to clean

In [28]:
# Mapping: variant name -> canonical name (first entry in each list)
NAME_CANON_MAP = {
    "24 St & 41 Ave (original)": "24 St & 41 Ave",

    "34th Ave & Vernon Blvd": "34 Ave & Vernon Blvd",

    "Central Park West & W 68 St": "Central Park W & W 68 St",
    "Central Park West & W 72 St": "Central Park W & W 72 St",
    "Central Park West & W 76 St": "Central Park W & W 76 St",
    "Central Park West & W 85 St": "Central Park W & W 85 St",

    "W 82 St & Central Park West": "W 82 St & Central Park W",
    "W 106 St & Central Park West": "W 106 St & Central Park W",

    "5 Ave & W 139 St": "5 Av & W 139 St",

    "W 168 St & Ft Washington Ave": "W 168 St & Fort Washington Ave",
    "W 180 St & Ft Washington Ave": "W 180 St & Fort Washington Ave",
}

df_citibike["start_station_name"] = (
    df_citibike["start_station_name"]
    .replace(NAME_CANON_MAP)
)

df_citibike["end_station_name"] = (
    df_citibike["end_station_name"]
    .replace(NAME_CANON_MAP)
)

the mapping id -> name is now ***injective***

## Station Coordinate Offset

In [29]:
def haversine_m(lat1, lon1, lat2, lon2) -> np.ndarray:
    """Vectorized haversine distance in meters."""
    R = 6_371_000.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    return 2.0 * R * np.arcsin(np.sqrt(a))


def station_offset_summary_by_name(df: pd.DataFrame) -> pd.DataFrame:
    """
    One output table:
      station_name | n_points | std_m | max_m

    Points = all start + end coordinate observations grouped by station_name.
    Distances are computed to a robust station center (median lat/lng).
    """
    pts = pd.concat(
        [
            df[["start_station_name", "start_lat", "start_lng"]].rename(
                columns={"start_station_name": "station_name", "start_lat": "lat", "start_lng": "lng"}
            ),
            df[["end_station_name", "end_lat", "end_lng"]].rename(
                columns={"end_station_name": "station_name", "end_lat": "lat", "end_lng": "lng"}
            ),
        ],
        ignore_index=True,
    )

    centers = pts.groupby("station_name", as_index=False).agg(
        center_lat=("lat", "median"),
        center_lng=("lng", "median"),
    )

    pts = pts.merge(centers, on="station_name", how="left")

    pts["dist_m"] = haversine_m(
        pts["lat"].to_numpy(),
        pts["lng"].to_numpy(),
        pts["center_lat"].to_numpy(),
        pts["center_lng"].to_numpy(),
    )

    out = (
        pts.groupby("station_name", as_index=False)
        .agg(
            n_points=("dist_m", "size"),
            std_m=("dist_m", "std"),
            max_m=("dist_m", "max"),
        )
        .sort_values("max_m", ascending=False, ignore_index=True)
    )

    # std can be NaN for stations with n_points == 1
    out["std_m"] = out["std_m"].fillna(0.0).round(1)
    out["max_m"] = out["max_m"].round(1)

    return out


offset_table = station_offset_summary_by_name(df_citibike)
display(offset_table)

,station_name,n_points,std_m,max_m
0,Kent Ave & N 3 St,477,69.500000,290.399994
1,Bridge St & Front St,2352,3.900000,190.000000
2,Soissons Landing,761,41.799999,95.099998
3,W 47 St & 6 Ave,1964,27.900000,56.700001
4,Seneca Ave & Irvine St,87,18.799999,37.599998
...,...,...,...,...
2229,Carroll St & 6 Ave,1854,0.000000,0.000000
2230,Carroll St & 5 Ave,1943,0.000000,0.000000
2231,Carmine St & 6 Ave,6206,0.000000,0.000000
2232,Carlton Ave & St Marks Ave,779,0.000000,0.000000


- there are some stations which probably got hard coded lat/long values and other stations which where maybe relocated/moved
- we could use the median to infere consistent coordinates for each station but that is neither needed nor useful for our project

## Conclusion

df_citibike:
- Nans removed
- internal stations (ID = SYS....) removed
- invalid coordinate rows removed
- unique station identifier Station name

df_citibike_missing:
- data frame which holds the Nans

Interesting information:
- data is for trips which ended in the timeframe,start time can be earlier
- data is for trips which startet in NYC, endlocation can be outside
- data originally included some trips to/from internal stations even though it is explicitly stated that staff trips where removed in the data description
- some stations have hard coded coordinates other have high offset which indicates faulty gps tracking or relocation

In [33]:
df_citibike.to_pickle(OUT_DIR / "citibike_clean.pkl")
df_citibike_missing.to_pickle(OUT_DIR / "citibike_missing.pkl")